1. Design Model
    - Input Size
    - Output Size
    - Forward Pass
1. Construct loss and parameters
1. Training loop
    1. Forward Pass: Computer prediction and loss
    1. Backward Pass: Gradients
    1. Update Weights

In [119]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [120]:
# prepare data
bc = datasets.load_breast_cancer()
x, y = bc.data, bc.target

nSamples, nFeatures = x.shape
print(f"Number of samples: {nSamples}")
print(f"Number of features: {nFeatures}")

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=1234)

Number of samples: 569
Number of features: 30


In [121]:
# scaling and casting data
sc = StandardScaler()
xTrain = sc.fit_transform(xTrain)
xTest = sc.transform(xTest)

xTrain = torch.from_numpy(xTrain.astype(np.float32))
xTest = torch.from_numpy(xTest.astype(np.float32))
yTrain = torch.from_numpy(yTrain.astype(np.float32))
yTest = torch.from_numpy(yTest.astype(np.float32))

yTrain = yTrain.view(yTrain.shape[0], 1)
yTest = yTest.view(yTest.shape[0], 1)

In [122]:
# model
# f = wx + b, sigmoid at the end

class LogisticRegression(nn.Module):
    def __init__(self, nInputFeatures):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(nInputFeatures, 1)

    def forward(self, x):
        yPredicted = torch.sigmoid(self.linear(x))
        return yPredicted
    
model = LogisticRegression(nFeatures)

In [123]:
# loss and optimizer
learningRate = 0.01

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [124]:
# training loop
numEpoch = 300

for epoch in range(numEpoch):
    # forward pass and loss
    yPredicted = model(xTrain)
    loss = criterion(yPredicted, yTrain)

    # backward pass
    loss.backward()

    # updates
    optimizer.step()

    #zero gradients
    optimizer.zero_grad()

    if epoch % 30 == 0: 
        print(f"epoch: {epoch+1}, loss = {loss.item():.4f}")


epoch: 1, loss = 0.8184
epoch: 31, loss = 0.4212
epoch: 61, loss = 0.3128
epoch: 91, loss = 0.2600
epoch: 121, loss = 0.2274
epoch: 151, loss = 0.2048
epoch: 181, loss = 0.1879
epoch: 211, loss = 0.1747
epoch: 241, loss = 0.1640
epoch: 271, loss = 0.1552


In [125]:
with torch.no_grad():
    yPredicted = model(xTest)
    yPredicted = yPredicted.round()
    acc = yPredicted.eq(yTest).sum() / float(yTest.shape[0]) #eq is equal to
    print(f"Accuracy = {acc:.4f}")

Accuracy = 0.9123
